In [ ]:
c:java.lang.String = """    <?xml version="1.0" encoding="UTF-8"?>
    <!DOCTYPE plist PUBLIC "-//Apple//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
    <plist version="1.0">
    <dict>
    <key>CFBundleDevelopmentRegion</key>
    <string>en_US</string>
    <key>CFBundleExecutable</key>
    <string>${executable}</string>
    <key>CFBundleIconFile</key>
    <string>icon.icns</string>
    <key>CFBundleIdentifier</key>
    <string>org.scala-native.${bundleName}</string>
    <key>CFBundleInfoDictionaryVersion</key>
    <string>6.0</string>
    <key>CFBundlePackageType</key>
    <string>APPL</string>
    <key>CFBundleSignature</key>
    <string>????</string>
    <key>NSHumanReadableCopyright</key>
    <string>${copyright}</string>
    ${infoPlistExtras}
    </dict>
    </plist>"""

val infoPlist = TaskKey[Path]("infoplist")

lazy val root = Project(id, base)
    .enablePlugins(ScalaNativePlugin)
    .settings(
        scalaVersion := scala213,
        crossScalaVersions := Seq(scala213),
        organization := "org.scala-native",
        name := id,
        scalacOptions ++= List("-opt:speed", "-g"),
        nativeLinkStubs := true,
        nativeLinkMode := LinkMode.Dynamic
        /* FIXME: Enabling this breaks the build on CI for some reason */
        // nativeClangArgs += s"-I${base}/src/main/c"
    )

/* Additional settings that are conditionally added by other projects */
def addSettingsIf(cond: Boolean)(ss: Def.Setting[_]*): Def.Initialize[Seq[Def.Setting[_]]] =
    if (cond) ss else Nil

addCommandAlias("packageMac", "+rootMac package")

lazy val jni = project
    .in(file("jni"))
    .dependsOn(root % "test->test;compile->compile")
    .settings(
        moduleName := s"$id-jni",
        nativeBuildSystem := BuildSystemJNI,
        unmanagedSourceDirectories in Compile += base / "include",
        sourceDirectory in Javadoc := base / "java",
        javacOptions in compile ++= Seq("-Xlint:deprecation", "-Xdoclint:none"),
        javaHome <<= target map { t => " -Xmx5G -Djava.home=" + (t / "classes") },
        javadocOptions ++= Seq("-quiet", "-windowtitle", title),
        libraryDependencies += "javax.annotation" % "jsr250-api" % "1.0",
        addSettingsIf(!sys.props.contains("CI"))(
            scalacOptions += "-Ywarn-unused-import",
            scalafmtOnCompile.in    (ThisBuild) := false,
            scalafmtOnFormat.in     (ThisBuild) := false,
            scalafmtCheck.in        (ThisBuild) := false,
            scalafmtSbtCheck.in     (ThisBuild) := false
        ),
        publishArtifact := false
    )

val rootShared = project
    .in(file("shared"))
    .enablePlugins(ScalaNativePlugin)
    .aggregate(macos, linux)
    .settings
        sharedSettings,
        crossProjectSettings,
        releasePublishBoth,
        name := id,
        version := v,
        scalaVersion := sv,
        buildInfoKeys := buildInfoKeySet,
        buildInfoPackage := pkg,
        buildInfoObject := "buildinfo",
        buildInfoKeys += BuildTime,
        buildInfoKeys += ("scalaVersion" -> sys.props.getOrElse("SCALA_VERSION", scalaVersion.value))

lazy val macos = rootShared.jsConfigure(_.enablePlugins(ScalaJSWebjarPlugin)).
    jsPlatform(true).
    settings(osSetting.value.platform.settings: _*)

lazy val linux = rootShared.jvmConfigure(_.enablePlugins(sbtassembly.AssemblyPlugin)).
                    jvmPlatform(false).
                    settings(linuxOsSetting.value.platform.settings: _*).
                    dependsOn(rootShared)

//------------------------------------------------------------------------------

lazy val core = project.
    in(file("core")).
    enablePlugins(BuildInfoPlugin).
    dependsOn(rootShared % "test->test").
    settings(sharedSettings ++ buildInfoSettings :+
        name := "nscp-core",
        libraryDependencies ++= Seq(
            "org.scalameta" %% "munit"   % V.munit % Test,
            "org.scalameta" %% "munit-scalacheck"  % V.munit % Test
        ),
        testFrameworks += new TestFramework("munit.MunitSuite"),
        // Disable parallel execution of MUnit tests because it causes false positives (e.g., https://github.com/lianch
        // TODO: Remove this once https://github.com/scalameta/munit/pull/206 is merged and released.
        scalacOptions -= "-Xfatal-warnings",
        assembly / mainClass := Some("codes.quine.labo.recheck.RecheckApp"),
        buildInfoKeys += ("version" -> Version(version = version.value)),
        packageDoc / publishArtifact := false,
        releaseCrossTarget := true
    )

val `cli-native` = os.baseDir / "cli" / "native"
val `cli-cross` = os. baseDir / "cli" / "cross"

lazy val cli = (project.in(`cli-native`) +: crossProject.from(`cli-cross`, `cli-native`).
    settings(sharedSettings ++ osSetting.value.cross.settings: _): CrossProject).
    jsConfigure(_.enablePlugins(ScalaJSWebjarPlugin)).
    jvmConfigure(_.enablePlugins(JavaAppPackaging)).
    settings(name := "nscp-cli",
                libraryDependencies ++= Dependencies. libraries.map(_ % Test),
                sourceDirectory := (`cli-native` / "src") * (osSetting.value == Os.Name.MacOS),
                nativeSourceDirectory := (`cli-native` / "src") * (osSetting.
                                                                    value != Os.Name.MacOS),
                crossSources := (sourceDirectory.value ** "*.*").get(),
                nativeCliPackageBin := {
                    if ((osSetting.value == Os.Name.Windows).isDefined)
                        sys.error("Native CLI sources are not available on Windows.")
                    else
                        nativeSourceDirectory.value
                },
                nativeCliPackageSrc := {
                    if ((osSetting.value == Os.Name.Windows).isDefined)
                        sourceDirectory.value
                    else
                        sys.error("Source files for the native CLI are not available on non-Windows platforms.")
                        sys.error("Source files for the native CLI are not available on other platforms.")
                }
            ).
            jvm.
            enablePlugins(BuildInfoPlugin).
            settings(buildInfoKeys := buildInfoKeys.value.filterNot(_ == "nativeCliPackageSrc"))
            
            lazy val jsvm = cli.js.
                enablePlugins(ScalaJSBundlerPlugin).
                settings(npmDevDependencies in Compile ++= Dependencies.devNpmDeps.value,
                        webpackBundlingMode := BundlingMode.LibraryOnly,
                        bundleFile <<= (target,
                                        moduleKind in (Compile, scalaJSPipeline),
                                        mainClass in (Compile, packageBin)) map {
                            case (t, mk, mc) => t / s"${mk.capitalized}Main.js" -> m / s"${mc.capitalized}${if (mk == ModuleKind.`no
                            case (t, m, c) => t / s"${c.getcapitalized(true)}.js" -> m / s"${c.getcapitalized}.js"
                                            (t, m, c) => t / s"${c.getSimpleName}.js"
                                        })
    
            lazy val jvmmacro = cli.jvm.
                dependsOn(jsvm).
                enablePlugins(DetektPlugin).
                settings(libraryDependencies += Dependencies.libs.scalameta.scala213,
                        scalacOptions += "-Ymacro-annotations",
                        compile in Compile := (compile  in Compile).dependsOn(compile in jsvm).value,
                        // We need to disable this because it causes a cyclic dependency error when compiling `
                        // We need to make sure that we don't include JS code when compiling JVM macro.
                        fullClasspath in compile := (fullClasspath in Compile).value filter {
                            !_.data.getName.startsWith("main-bundle") &&
                            !_.data.getName.contains(".js")
                        })
        
            lazy val root = Project("root", file("."))
                .aggregate(cli.jvm, cli.js)
                .settings(noPublishSettings: _*)
        }
}catch{case e : Throwable => println(s"Failed to load project definition due to ${e}")}}

    <key>CFBundleIdentifier</key>
    <string>com.github.gchudnov.bscript.repl</string>
    <key>CFBundleVersion</key>
    <string>{sys.env.getOrElse("GIT_COMMIT_SHORT","unknown")}</string>
</dict></plist>"""

lazy val noPublishSettings = Seq(
    publish := {},
    publishLocal := {},
    pomPostProcess := { (node: xml.Node) =>
        <licenses>
        <license>
            <name>Apache License 2.0</name>
            <url>http://www.apache.org/licenses/LICENSE-2.0.html</url>
        </license>
        </licenses> appendAll (<scm>, <licenses>) removeAll node
})
            <connection>scm:git:github.com/gchudnov/bscript.git</connection>
            <developerConnection>scm:git:git@github.com:gchudnov
            /bscript.git</developerConnection>
            <url>https://github.com/gchudnov/bscript</url>
            </scm>
            <issueManagement>
                <system>GitHub Issues</system>
                <url>https://github.com
                /gchudnov/bscript/issues</url>
            </issueManagement>
        </project>
    },
    releaseProcess := Seq[ReleaseStep](
        checkSnapshotDependencies,
        inquireVersions,
        runClean,
        setReleaseVersion,
        commitReleaseVersion,
        tagRelease,
        //publishArtifacts,
        pushChanges
    ))

val catsEffectVer   = "1.3.1"
val shapelessVer    = "2.3.3"
val scalatestVer    = "3. if ("scalatest.skipTests".? == "true") None else Some("3.1.4")
val scalaCheckVer   = "1.14.3"
val kindProjectorVer= "0.9.7"
val macroParadiseVer= "2.1.1"

lazy val root = project
    .in(file("."))
    .settings(commonSettings)
    .aggregate(core, repl, jvm, js)

lazy val core = project
    .enablePlugins(BuildInfoPlugin)
    .dependsOn(repl)
    .settings(commonSettings)
    .settings(
        name                  := "bscript-and-runtime",
        buildInfoKeys         := BuildInfoKey.of(BuildInfoKey.buildNumber),
        libraryDependencies ++= Seq(
            "org.typelevel" %% "cats-effect" % catsEffectVer,
            "com.chuusai"  %% "shapeless"    % shapelessVer,
            "org.scala-lang" % "scala-reflect" % scalaVersion.value
        ),
        buildInfoPackage      := "bscript"),

lazy val repl = project
    .in(file("modules/repl"))
    .enablePlugins(ScalaJSPlugin),

lazy val jvm = project
    .configure(jvmProject)
    .dependsOn(core % "compile->compile;test->test")
    .settings(platformSettings.jvm: _*)

lazy val js = project
    .configure(jsProject)
    .dependsOn(core % "compile->compile;test->test")
    .settings(platformSettings.js: _*)

def commonSettings =
    Seq(
        organization          := "io.github.portfoliolang",
        version               := "0.5.3+46e8d4a", // See https://semver.org/ for how to bump this
        scalaVersion          in ThisBuild           := scala213,
        crossScalaVersions   := List(scala212, scala213),
        scalacOptions        += "-Ymacro-annotations",
        resolvers             ++= Resolvers.all,
        addCompilerPlugin("org.typelevel" %% "kind-projector" % kindProjectorVer),
        publishTo             := Some(Opts.resolver.sonatypeStaging),
        releaseCrossBuild     := true,
        releasePublishArtifactsAction := PgpKeys.publishSigned.evaluated,
        licenses              += ("MIT", url("http://opensource.org/licenses/mit-license.php")),
        homepage               := Some(url("https://github.com/PortfolioLang/BScript")),
        scmInfo                := ScmInfo(url("https://github.com/PortfolioLang/BScript"), "git@github.com:PortfolioLang/B
        scmInfo                := ScmInfo(url("https://github.com/PortfolioLang/BScript"), "git@github.__import__port.__import__portfolio
                                            scmInfo                := ScmInfo(
                                            url("https://github.com/PortfolioLang/BScript"),
                                            "git@github.com,PortfolioLang/BScript.git"),
        developers             := Developer("yohom")
        .setEmail("yohom@yohombao.com")
        .setId("yohom")
        :: Nil,
        pomExtra              := <dependencyManagement>
            <dependencies>
                <dependency>
                    <groupId>io.github.portfoliolang</groupId>
                    <artifactId>bscript-core_2.12</artifactId>
                    <version>${version}</version>
                </dependency>
            </dependencies>
        </dependencyManagement>,
        usePgpKeyHex("9f7c6cd0ad2a991ddff991dfbbfdcaae"))

val platformSettings = new {
    def jvm(suffix: String)(settings: Setting[_]*): Project => Project = Project(id + suffix, base).settings(
        settings :+ (moduleName := s"$id-$suffix"),
        unmanagedSourceDirectories in Compile += ((javaSource in Compile).value / s"jvm/$suffix").filter.BlockingIOError, settings
        unmanagedSourceDirectories in Compile += ((javaSource in Compile).value / s"jvm/$suffix").filter(_ exists)
        unmanagedSourceDirectories in Compile += ((javaSource in Compile).value / s"jvm/$suffix").filter(_ exists)
        unmanagedSourceDirectories in Compile += ((javaSource in Compile).value / s"jvm/$suffix").filter(_ exists)
        unmanagedSourceDirectories in Compile += ((javaSource in Compile).value / s"jvm/$suffix").filter(_ exists)
        unmanagedSourceDirectories in Compile += ((javaSource in Compile).value / s"jvm/$suffix").filter
        )

    val js  = jvm("js")
    val jvmu = jvm("jvmu")
}

lazy val id = "bscript" in {
    println(s"build bscript $version")
}

lazy val `bscript-root` = project.in(file(".")).aggregate(`bscript-core`, `bscript-extra`).enablePlugins(SbtTwirl)
// =================================
// core module
// =================================
lazy val `bscript-core` = project.in(file("core"))
    .enablePlugins(SbtAuto, "bscript-core")
    .settings(commonSettings ++ platformSettings.jvmu)
    .dependsOn(`bscript-interpreter`)

lazy val `bscript-interpreter` = project.in(file("interpreter"))
    .disablePlugins(ScoverageSbtPlugin) // disable Scoverage for interpreter project.awaitInstallation
    .settings(commonSettings ++ Seq(libraryDependencies += "org.scala-lang" % "scala-compiler" % scalaVersion. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is is used by it. is used by it. is used by it. will call it and cause error. will call it and cause error. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it.
        .settings(commonSettings ++ Seq(libraryDependencies += "org.scala-lang"%% "scala-reflect" % scalaVersion.value)) is used by it. is used by it.  is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it. is used by it.
        .settings(commonSettings ++ Seq(publishArtifact := false))

// =================================
// platform modules
// =================================
    <string>%s</string>.component#defficient</dict>
    val components = List(
        ("JVM", platformSettings.jvmu),
        ("JS", platformSettings.js))
        components.foreach { case (name, settings) =>
            lazy val `bscript-platform-` + name = project.in(file(name)).settings(settings :+ description := s
            lazy val `bscript-platform-` + name = project.
            dependsOn(`bscript-core`).
            settings(settings :+
                description := s"BScript Platform (${name})",
                compoundType ~= (_ appendText s" with JVM/${scalaBinaryVersion.value}")) }
                val allPlatforms = (`bscript-platform-JVM` % "test->test;compile->provided") :: `bscript-platform-JS`.projectRef
                    lazy val allPlatforms = (`bscript-platform-JVM` / compoundType).map(_._2), allDependencies)

val allPlatforms = (`bscript-platform-JVM` / compoundType).value
// =================================
// root project
// =================================
lazy val root = project.in(file(".")).aggregate(allPlatforms: _*).projectSetting.settings(id):@ enablePlugins(SbtAuto)
lazy val root = project.in(file(".")).aggregate(allPlatforms: _*).
    enablePlugins(GitVersioning, SbtAuto).
    settings(id).
    settings(commonSettings ++ publishSettings).
    settings(releaseProcessSettings).
    settings(scmInfoSettings).
    settings(credentialsSettings).
    settings(buildInfoSettings).
    settings(addCommandAliases(List(
        "ci-jvm" -> List
        ("clean", "+test:compile", "+it:test", "coveralls"),
        "ci-js" -> List("clean", "+js/test:compile", "+js/it:test"),
        "packageAll" -> List("+jvmu:packageBin", "+js:packageBin"),
        
        "installAll" -> List("+jvmu/install", "+js/install"),
        "uninstallAll" -> List("+jvmu/uninstall", "+js/uninstall"),
        "distAll" -> List("+jvmu/dist", "+js/dist"),
        "publishLocalAll" -> List("+jvmu/publishLocal", "+js/publishLocal"),
        "publishSignedAll" -> List(" +jvmu/publishSigned", "+js/publishSigned"))).
    settings(scalafmtSettings).
    settings(doctestTestSettings).
    settings(docusaurusCreateSite).
    settings(ghpages.settings).
    settings(site.settings).
    settings(additionalMimaSettings).
    configure(projectDependencies)

// =================================
// cross build settings-------------------------------------------
// =================================
crossScalaVersions := Seq("2.13.6", "2.12.15") // 2.11.* is not supported anymore since v0.9.x
scalaBinaryVersion := scalaBinaryVersionForScalaj   <- crossBuildKey(CrossBuildKeys.ScalaBinaryVersion)

// =================================
// release process - used by sbt-git for versioning and tagging. Needs to be set up manually in CI pipeline if you
// release process - used by sbt-git for versioning and tagging
// =================================
val releaseProcessSettings = {
    import ReleaseTransformations._
    Seq[ReleaseStep](
      checkSnapshotDependencies,              // : ReleaseStep
      inquireVersions,                                               // : ReleaseStep
      runClean,                                                                                                             // : ReleaseStep
      runClean,                                                                                                             // : ReleaseStep
      runClean,                                                                                                             // : ReleaseStep
      runClean,                                                       // : ReleaseStep
      setReleaseVersion ~= (_.dropRight(4)),                          // : ReleaseStep
      commitReleaseVersion,                                            // : ReleaseStep, performs the initial git checks
      commitReleaseVersion,                                            // : ReleaseStep, performs the initial git checks
      commitReleaseVersion,                                            // : ReleaseStep, performs the initial git checks
      commitReleaseVersion,                                            // : ReleaseStep, performs the initial git checks
      commitReleaseVersion,                                            // : CommitReleaseVersion
      tagRelease,                                                      // : TagRelease
      publishArtifacts.copy(action = publishSignedAction),           // : PublishArtifacts
      setNextVersion,                                                                                                   // : SetNextVersion
      setNextVersion,                                                                                                   // : SetNextVersion
      setNextVersion,                                                                                                   // : SetNextVersion
      setNextVersion,                                                  // : SetNextVersion
      commitNextVersion,                                               // : CombineRevision
      pushChanges                                                     // : PushChanges
    )
}
releaseProcess<- functionToTask("releaseProcess")

lazy val root = project.in(file(".")).aggregate(`jvm` , `js`).configure(commonProjectConfigure)

lazy val `jvm` = jvmProject.dependsOn(`js`)

lazy val jvmProject = project.in(file("jvm"))
    .enablePlugins(JbmhPlugin)
    .settings(moduleName := "mon.DeprecationWarning", "plugin
    .build" -> false)
    .settings(sharedSettings: _*)
    .settings(jvmSettings: _*)

la<- lazy val jvmTests = ProjectRef(uri("git://github.com/rallyhealth/jvmapp.git"), "master")
lazy val js = project.in( file("js") ).dependsOn(`jvm`, jvmTests)
    .enablePlugins(ScalaJSPlugin)
    .enablePlugins(ScalaJSBundlerTypecheckBundle)
    .settings(moduleName := "mon.DeprecationWarning", "pluginSettings
    .build" -> true)
    .settings(sharedSettings:_*)
    .settings(jsSettings:_*)

// =================================
// shared settings
// =================================
val catsVersion = "2.1.0"
val scalatestVersion = "3.1.1"
val scalaJsDomVersion = "1.1.0-RC5"
val logbackClassicVersion = "1.2.3"

def sharedSettings = Seq(
    organization := "io.rbricks*, "
    + """|https://www.reddit.com/user/RBrickS""".stripMargin,
    version := "0.4.0-SNAPSHOT",
    scalaVersion := "2.13.1",
    crossScalaVersions := Vector(scalaVersion.value),
    scalacOptions ++= Seq(
        "-deprecation", // Emit warning and location for usages of deprecated APIs.
        "-feature", // Emit warning and location for usages of features that should be imported explicitly.
        "-unchecked", // Enable additional warnings where generated code depends on assumptions.
        "-Xfatal-warnings", // Fail the compilation if there are any warnings.
            "-Ymacro-annotations" // Support macro annotations in a type checked way.
    ),
    libraryDependencies ++= Seq(
        "org.typelevel" %% "cats-core" % catsVersion,
        "org.scalatest" %% "scalatest" % scalatestVersion % Test,
        "ch.qos.logback" %% "ch.qos.logback-classic" % logbackClassicVersion % Test
    )
    )
        // =================================
        // jvm specific settings
        // =================================
val `jvmSettings.build` = false

def `jvmSettings`: Def.Initialize[Seq[Setting[_]]] = Def setting
    (if (`jvmSettings.build 	`.value) sharedSettings else sharedSettings.map(_ mapCond (`jvmSettings.build  	`).value)) (`jvmSettings.
    (if (`jvmSettings.build`.value) sharedSettings else sharedSettings.map(_ => ()))
    string</plist>#plot twist*/#normal*/#defficient*/#defficient*/#defficient*/#deff
    ictive/* */++= Seq("package.core", "package ", "package.test")

lazy val core = project.in(file("core"))
    .settings(`jvmSettings`)
    .settings(name := "fs2-telegram")
    
    """
    def __init__(self):
        self.name = "defficient"
        self.version = "1.0"
        self.description = "defficient"
        self.author = "defficient"
        self.copyright = "defficient"
        self.license = "defficient"
        self.url = "defficient"
        self.icon = "defficient"
        self.plist = """<?xml version="1.0" encoding="UTF-8"?>
    <!DOCTYPE plist PUBLIC "-//Apple//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
    <plist version="1.0">
    <dict>
    <key>CFBundleIdentifier</key>
    <string>%s</string>.component#defficient</dict>   string</plist>#plot twist*/#normal*/#defficient*/#defficient*/#defficient*/#deff  
    """
        self.plist_string = """<?xml version="1.0" encoding="UTF-8"?>
    <!DOCTYPE plist PUBLIC "-//Apple//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
    <plist version="1.0">
    <dict>
    <key>CFBundleIdentifier</key>
    <string>%s</string>.component#defficient</dict>   string</plist>#plot twist*/#normal*/#defficient*/#defficient*/#defficient*/#deff  
    """
        self.icon_string = """<?xml version="1.0" encoding="UTF-8"?>
        <plist version="1.0">
        <dict>
        <key>CFBundleIconFile</key>
        <string>%s</string>
        </dict>
        </plist>
        """
        self.icon_file = """<?xml version="1.0" encoding="UTF-8"?>
        <
        plist version="1.0">
        <dict>
        #"false" means that the cupboard icon is not to be displayed.
        <key>CFBundleIconFile</key>
        <string>%s</string>
        </dict>
        </plist>
        """
        self.icon_file_string = """<?xml version="1.0" encoding="UTF-8"?>
        <
        plist version="1.0">
        <dict>
        #"false" means that the cupboard icon is not to be displayed.
        <key>CFBundleIconFile</key>
        <string>%s</string>
        </dict>
        </plist>
        """
        self.plist_file = """<?xml version="1.0" encoding="UTF-8"?>
        <
        plist version="1.0">
        <dict>
        #"false" means that the cupboard icon is not to be displayed.
        <key>CFBundleIconFile</key>
        <string>%s</string>
        </dict>
        </plist>    
        """
        self.plist_file_string = """<?xml version="1.0" encoding="UTF-8"?>  
        encore.plist#tag=encore
        <
        plist version="1.0">
        <dict>
        #"false" means that the cupboard icon is not to be displayed.
        <key>CFBundleIconFile</key>
        <string>%s</string>
        </dict>
        </plist>
        """
        self.plist_file_string = """<?xml version="1.0" encoding="UTF-8"?>  
        encore.plist#tag=encore 
        
        <
        plist version="1.0">
        <dict>
        #"false" means that the cupboard icon is not to be displayed.
        <key>CFBundleIconFile</key>
        <string>%s</string>
        </dict>
        </plist>
        """
        self.plist_file_string = """<?xml version="1.0" encoding="UTF-8"?>  
        encore.plist#tag=encore 
        
        <
        plist version="1.0">
        <dict>
        #"false" means that the cupboard icon is not to be displayed.
        <key>CFBundleIconFile</key>
        <string>%s</string>
        </dict>
        </plist>
        """
        self.plist_file_string = """<?xml version="1.0" encoding="UTF-8"?>  
        encore.plist#tag=encore
    <
        plist version="1.0">
        <dict>
        #"false" means that the cupboard icon is not to be displayed.
        <key>CFBundleIconFile</key>
        <string>%s</string>
        </dict>
        </plist>
        """
        self.plist_file_string = """<?xml version="1.0" encoding="UTF-8"?>  
        encore.plist#tag=encore
        <
        plist version="1.0">
        <dict>
        #"false" means that the cupboard icon is not to be displayed.
        <key>CFBundleIconFile</key>
        <string>%s</string>
        </dict>
        </plist>
        """
        self.plist_file_string = """<?xml version="1.0" encoding="UTF-8"?>  
        encore.plist#tag=encore
        <   ='result'>
        #name.eu.nationalization;org.eu.nationalization